# <center>**Space X Falcon 9 First Stage Landing Prediction**</center>

## Interactive Visual Analytics with Folium

In [189]:
import folium
import pandas as pd

In [190]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [191]:
# The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude
# added for each site.
df = pd.read_csv('../data/spacex_launch_geo.csv')
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [192]:
df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [193]:
df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_site_df = df.groupby('Launch Site', as_index=False).first()
launch_site_df = launch_site_df[['Launch Site', 'Lat', 'Long']]
launch_site_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [194]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.

In [195]:
# Create a circle with a radius of 3000 meters centered on the NASA Johnson Space Center's coordinates
circle = folium.Circle(nasa_coordinate, radius=3000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    popup='NASA Johnson Space Center',
    icon = folium.Icon(color='green', icon_color='white', popup = 'NASA JSC')
)

site_map.add_child(circle)
site_map.add_child(marker)

Display all launch sites

In [196]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i, launch_site in enumerate(launch_site_df['Launch Site']):
    site_coordinates = [launch_site_df['Lat'][i], launch_site_df['Long'][i]]
    circle = folium.Circle(site_coordinates, radius=1000, color='#000000', fill=True).add_child(folium.Popup(launch_site_df['Launch Site'][i]))
    marker = folium.map.Marker(
        site_coordinates,
        icon = DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_df['Launch Site'][i])
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

Proximity to the Equator Line

Looking at the latitude coordinates of the launch sites:
- CCAFS LC-40: 28.562302°N
- CCAFS SLC-40: 28.563197°N  
- KSC LC-39A: 28.573255°N
- VAFB SLC-4E: 34.632834°N

**Analysis**: Most launch sites are indeed in proximity to the Equator line. Three of the four sites (CCAFS LC-40, CCAFS SLC-40, and KSC LC-39A) are located at approximately 28°N latitude, which is relatively close to the Equator. This strategic positioning takes advantage of Earth's rotational speed, providing additional velocity boost for launches heading eastward. However, VAFB SLC-4E is located at 34.6°N, which is further from the Equator but still within reasonable range for polar orbit launches.

Proximity to the Coast

Examining the longitude coordinates and geographic context:
- CCAFS LC-40: -80.577356°W (Cape Canaveral, Florida)
- CCAFS SLC-40: -80.576820°W (Cape Canaveral, Florida)
- KSC LC-39A: -80.646895°W (Kennedy Space Center, Florida)
- VAFB SLC-4E: -120.610745°W (Vandenberg Air Force Base, California)

**Analysis**: All launch sites are in very close proximity to the coast. The three Florida-based sites (CCAFS and KSC) are located on the Atlantic coast, while VAFB SLC-4E is situated on the Pacific coast of California. This coastal positioning is intentional for safety reasons - it allows rocket stages and debris to fall into the ocean rather than populated areas, minimizing risk to human life and property during launch operations.

Strategic Implications
The combination of equatorial proximity and coastal location makes these sites ideal for space launches, balancing the benefits of Earth's rotation with safety considerations for launch operations.


We create a new column in <code>spacex_df</code> dataframe called <code>marker_color</code> to store the marker colors based on the class value

In [197]:
df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [198]:
marker_color = []
for i in range(len(df)):
    if df['class'][i] == 1:
        marker_color.append('green')
    else:
        marker_color.append('red')

df['marker_color'] = marker_color
df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


Create a marker cluster

In [199]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    icon = folium.Icon(color = 'white', icon_color = record['marker_color'], icon='rocket', prefix='fa')
    marker = folium.map.Marker(coordinate, radius=1000, color='#000000', fill=True)
    marker_cluster.add_child(marker)
    marker_cluster.add_child(icon)

site_map

In [200]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

# Loop through each launch record
for index, record in df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    
    # Determine marker color based on success/failure
    if record['class'] == 1:
        icon_color = 'green'
    else:
        icon_color = 'red'

    # Create a marker with customized icon
    marker = folium.Marker(
        location = coordinate,
        icon = folium.Icon(color='white', icon_color=icon_color, icon='rocket', prefix='fa'),
        popup=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}"
    )

    marker_cluster.add_child(marker)

# Display the map
site_map

#### Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map

In [201]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [202]:
launch_site_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [203]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [204]:
coastline_coordinates = [34.63489, -120.62522]
distance_coastline = calculate_distance(df['Lat'][3], df['Long'][3], coastline_coordinates[0], coastline_coordinates[1])
distance_coastline

3828.3611923590206

Visualize the distance on the map

In [205]:
# visualize the distance from coastline to launch site
coastline_marker = folium.Marker(
    coastline_coordinates,
    popup=f'Coastline Point<br>Distance to Launch Site: {distance_coastline:.2f} km',
    icon=folium.Icon(color='blue', icon='info-sign')
)

line = folium.PolyLine(locations=[(34.63489, -120.62522), (launch_site_df['Lat'][3], launch_site_df['Long'][3])], line_opacity=0.5)
site_map.add_child(line)
site_map.add_child(coastline_marker)

site_map